# Building Neural Network from Scratch Part - 7

### Calculating Network Error With Loss

In [1]:
# Importing Numpy
import numpy as np
from nnfs.datasets import spiral_data
import nnfs
import matplotlib.pyplot as plt
nnfs.init()

### Case - 1: When The Class Targets are Just Numbers

[0, 1, 1] means [Red, Green, Green]

In [2]:
# A 2D NumPy array containing softmax output probabilities
softmax_outputs = np.array([
    [0.7, 0.1, 0.2],    # Sample 0: Class 0 has highest probability
    [0.1, 0.5, 0.4],    # Sample 1: Class 1 is the target
    [0.02, 0.9, 0.08]   # Sample 2: Class 1 has highest probability
])

# Actual class labels (targets) for each sample
class_targets = np.array([0, 1, 1])
# print(len(class_targets.shape))    # 1

# softmax_outputs[[0, 1, 2], class_targets] grabs:
# - softmax_outputs[0, 0]
# - softmax_outputs[1, 1]
# - softmax_outputs[2, 1]
print(softmax_outputs[[0, 1, 2], class_targets])

[0.7 0.5 0.9]


In [3]:
# Extract the probability values corresponding to the correct classes
correct_confidences = softmax_outputs[range(len(softmax_outputs)), class_targets]
# Apply the negative log to get the cross-entropy loss for each sample
neg_log = -np.log(correct_confidences)
print(neg_log)

# Calculate the average loss across all samples (mean cross-entropy loss)
average_loss = np.mean(neg_log)
print(average_loss)

[0.35667494 0.69314718 0.10536052]
0.38506088005216804


### Case - 2: When The Class Targets are One Hot Encoded

In [4]:
true_y_value = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 1, 0]
])
# print(len(true_y_value.shape))    # 2

y_pred_clipped_value = np.array([
    [0.7, 0.2, 0.1],
    [0.1, 0.5, 0.4],
    [0.02, 0.9, 0.08]
])


# Extract the probability values corresponding to the correct classes
correct_confidences = np.sum(true_y_value * y_pred_clipped_value, axis = 1)
neg_log = -np.log(correct_confidences)
print(neg_log)

# Calculate the average loss across all samples (mean cross-entropy loss)
average_loss = np.mean(neg_log)
print(average_loss)

[0.35667494 0.69314718 0.10536052]
0.38506088005216804


### Implementing The Categorical Cross Entropy Class

In [5]:
# Base class for all loss functions
class Loss:
    
    # Calculates the mean loss over a batch of samples
    def calculate(self, output, y):
        
        # Perform the forward pass (implemented in derived class, e.g., CategoricalCrossEntropy)
        # This returns individual loss values for each sample in the batch
        sample_losses = self.forward(output, y)

        # Compute the average loss across all samples
        data_loss = np.mean(sample_losses)

        # Return the scalar average loss
        return data_loss


In [6]:
# Cross-entropy loss for classification tasks
# Inherits from a base Loss class
class Loss_CategoricalCrossEntropy(Loss):
    
    # Forward pass to compute the loss for each sample
    def forward(self, y_pred, y_true):
        
        # Get the number of samples in the batch
        no_samples = len(y_pred)

        # Clip predicted values to avoid log(0), which is undefined
        # Clipping prevents extremely small or large values that could cause numerical issues
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Case 1: Labels are integer class indices (e.g., [0, 2, 1])
        if len(y_true.shape) == 1:
            # Extract the confidence (probability) for the correct class for each sample
            # For each sample index `i`, get y_pred_clipped[i, y_true[i]]
            correct_confidences = y_pred_clipped[range(no_samples), y_true]

        # Case 2: Labels are one-hot encoded (e.g., [[1, 0, 0], [0, 1, 0]])
        elif len(y_true.shape) == 2:
            # Element-wise multiplication of predicted probs and true labels
            # Since only the correct class has a 1, this effectively selects the correct class confidence
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        # Compute the negative log of the correct class probabilities
        # This gives the cross-entropy loss for each sample
        negative_log_likelihoods = -np.log(correct_confidences)

        # Return the loss per sample (not averaged)
        return negative_log_likelihoods

In [7]:
# Softmax output predictions from a neural network
# Each row corresponds to the predicted probability distribution for one sample
softmax_outputs = np.array([
    [0.7, 0.1, 0.2],    # Sample 0: most confident in class 0
    [0.1, 0.5, 0.4],    # Sample 1: most confident in class 1
    [0.02, 0.9, 0.08]   # Sample 2: most confident in class 1
])

# True labels in one-hot encoded format
# Each row has '1' at the correct class index
class_targets = np.array([
    [1, 0, 0],  # Sample 0: true class is 0
    [0, 1, 0],  # Sample 1: true class is 1
    [0, 1, 0]   # Sample 2: true class is 1
])

# Create an instance of the cross-entropy loss function
loss_function = Loss_CategoricalCrossEntropy()

# Calculate the average loss across all samples
# This calls .forward() internally and returns a scalar
total_loss = loss_function.calculate(softmax_outputs, class_targets)

# Print the final averaged cross-entropy loss
print(total_loss)

0.38506088005216804


### Full Code Upto This Point